### Initial data read-in/prep

In [1]:
# Update sklearn 

# !pip install sklearn --upgrade

In [39]:
# Update tensorflow. 

# !pip install tensorflow --upgrade

In [36]:
# Import dependencies

from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder, MinMaxScaler,StandardScaler
from sklearn import tree
from sklearn.datasets import load_iris
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
from numpy import asarray
import os
%matplotlib inline
import matplotlib.pyplot as plt

In [37]:
from tensorflow.keras.utils import to_categorical

AttributeError: module 'tensorflow.python._pywrap_tensorflow_internal' has no attribute 'IsGoogleCudaEnabled'

In [4]:
# Read the census csv into a pandas DataFrame

census = pd.read_csv('../Resources/Tableau_clean/census_all.csv')
census.head()

,State,year,Name,Population,HHI,PovertyRate
0,1,2011,Alabama,4747424,42934,17.133186
1,2,2011,Alaska,700703,69014,9.292239
2,4,2011,Arizona,6337373,50752,15.835820
3,5,2011,Arkansas,2895928,40149,17.846507
4,6,2011,California,36969200,61632,14.096818


In [5]:
# rename State column for joining

census = census.rename(columns={'Name': 'state_name'})
census

,State,year,state_name,Population,HHI,PovertyRate
0,1,2011,Alabama,4747424,42934,17.133186
1,2,2011,Alaska,700703,69014,9.292239
2,4,2011,Arizona,6337373,50752,15.835820
3,5,2011,Arkansas,2895928,40149,17.846507
4,6,2011,California,36969200,61632,14.096818
...,...,...,...,...,...,...
250,51,2015,Virginia,8256630,65015,11.164628
251,53,2015,Washington,6985464,61062,13.005750
252,54,2015,West Virginia,1851420,41751,17.466809
253,55,2015,Wisconsin,5742117,53357,12.614651


In [6]:
# Read the medicare csv into a pandas DataFrame

medicare = pd.read_csv('../Resources/Tableau_clean/medicare_all.csv')
medicare

,state_id,year,state_name,beneficiaries_part_b,one_ambulatory_visit,diabetic_enrollees_age_65_to_75,average_diabetic_enrollees_hemoglobin_a1c_test,average_diabetic_enrollees_eye_exam,average_diabetic_enrollees_blood_lipids_test,average_female_enrollees_age_67_to_69,average_female_age_67_to_69_mammogram,beneficiaries_part_a_eligible,leg_amputations_per_1000_enrollees,discharges_ambulatory_care_sensitive_conditions_per_1000_enrollees
0,1.0,2011,Alabama,478784.0,82.5,69691.0,83.9,63.3,80.3,42267.0,62.8,501422.0,0.99,76.4
1,2.0,2011,Alaska,49648.0,70.9,5449.0,74.5,55.8,66.9,5151.0,55.3,54928.0,0.81,53.1
2,3.0,2011,Arizona,465298.0,78.2,50991.0,78.5,66.1,75.8,43614.0,64.3,501103.0,0.57,51.4
3,4.0,2011,Arkansas,327939.0,80.7,40202.0,82.5,66.1,76.2,29290.0,58.3,345431.0,0.85,77.0
4,5.0,2011,California,2238140.0,72.7,243999.0,80.6,64.1,78.0,190971.0,59.0,2378472.0,0.63,49.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,49.0,2015,Virginia,832699.0,83.3,98165.0,87.6,70.0,82.0,83135.0,64.5,703266.0,0.53,42.8
251,50.0,2015,Washington,610922.0,78.5,56474.0,86.6,67.7,76.3,56238.0,60.0,510796.0,0.46,32.7
252,51.0,2015,West Virginia,206961.0,80.8,29239.0,84.7,59.9,78.2,20777.0,59.1,171837.0,0.90,75.0
253,52.0,2015,Wisconsin,474364.0,79.8,46596.0,91.0,69.7,81.6,44595.0,71.9,418646.0,0.65,45.0


In [7]:
# Identify column names for joining

list(medicare.columns) 

['state_id',
 'year',
 'state_name',
 'beneficiaries_part_b',
 'one_ambulatory_visit',
 'diabetic_enrollees_age_65_to_75',
 'average_diabetic_enrollees_hemoglobin_a1c_test',
 'average_diabetic_enrollees_eye_exam',
 'average_diabetic_enrollees_blood_lipids_test',
 'average_female_enrollees_age_67_to_69',
 'average_female_age_67_to_69_mammogram',
 'beneficiaries_part_a_eligible',
 'leg_amputations_per_1000_enrollees',
 'discharges_ambulatory_care_sensitive_conditions_per_1000_enrollees']

In [8]:
# join csvs

data_all = pd.merge(census,
                 medicare[['state_id', 'year', 'state_name', 'beneficiaries_part_b', 'one_ambulatory_visit', 'diabetic_enrollees_age_65_to_75',
 'average_diabetic_enrollees_hemoglobin_a1c_test', 'average_diabetic_enrollees_eye_exam', 'average_diabetic_enrollees_blood_lipids_test',
 'average_female_enrollees_age_67_to_69', 'average_female_age_67_to_69_mammogram', 'beneficiaries_part_a_eligible', 'leg_amputations_per_1000_enrollees',
 'discharges_ambulatory_care_sensitive_conditions_per_1000_enrollees']],
                 on='state_name')
data_all

,State,year_x,state_name,Population,HHI,PovertyRate,state_id,year_y,beneficiaries_part_b,one_ambulatory_visit,diabetic_enrollees_age_65_to_75,average_diabetic_enrollees_hemoglobin_a1c_test,average_diabetic_enrollees_eye_exam,average_diabetic_enrollees_blood_lipids_test,average_female_enrollees_age_67_to_69,average_female_age_67_to_69_mammogram,beneficiaries_part_a_eligible,leg_amputations_per_1000_enrollees,discharges_ambulatory_care_sensitive_conditions_per_1000_enrollees
0,1,2011,Alabama,4747424,42934,17.133186,1.0,2011,478784.0,82.5,69691.0,83.9,63.3,80.3,42267.0,62.8,501422.0,0.99,76.4
1,1,2011,Alabama,4747424,42934,17.133186,1.0,2012,492195.0,82.8,72392.0,84.2,63.2,80.7,42535.0,62.7,517526.0,0.90,71.5
2,1,2011,Alabama,4747424,42934,17.133186,1.0,2013,498123.0,83.3,74492.0,84.9,63.8,81.8,44502.0,62.7,525015.0,0.87,65.4
3,1,2011,Alabama,4747424,42934,17.133186,1.0,2014,506023.0,83.2,76238.0,85.0,63.7,81.1,48751.0,62.8,534296.0,0.83,61.1
4,1,2011,Alabama,4747424,42934,17.133186,1.0,2015,510586.0,83.3,68202.0,86.1,64.0,81.0,52169.0,63.1,404987.0,0.79,62.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1270,56,2015,Wyoming,579679,58840,11.212240,53.0,2011,61634.0,73.0,5750.0,74.7,64.7,58.7,5517.0,57.4,65932.0,0.62,55.2
1271,56,2015,Wyoming,579679,58840,11.212240,53.0,2012,64431.0,73.3,6017.0,74.7,63.8,59.4,5519.0,57.9,69056.0,0.46,52.7
1272,56,2015,Wyoming,579679,58840,11.212240,53.0,2013,68066.0,74.6,6475.0,75.4,62.7,60.2,5844.0,57.1,73043.0,0.67,47.9
1273,56,2015,Wyoming,579679,58840,11.212240,53.0,2014,70457.0,73.8,6832.0,76.7,63.2,59.7,6498.0,56.3,75824.0,0.38,46.1


### Set features

In [9]:
# Get list of column names

list(data_all.columns) 

['State',
 'year_x',
 'state_name',
 'Population',
 'HHI',
 'PovertyRate',
 'state_id',
 'year_y',
 'beneficiaries_part_b',
 'one_ambulatory_visit',
 'diabetic_enrollees_age_65_to_75',
 'average_diabetic_enrollees_hemoglobin_a1c_test',
 'average_diabetic_enrollees_eye_exam',
 'average_diabetic_enrollees_blood_lipids_test',
 'average_female_enrollees_age_67_to_69',
 'average_female_age_67_to_69_mammogram',
 'beneficiaries_part_a_eligible',
 'leg_amputations_per_1000_enrollees',
 'discharges_ambulatory_care_sensitive_conditions_per_1000_enrollees']

In [23]:
features = data_all[['Population', 'PovertyRate', 'beneficiaries_part_b', 'one_ambulatory_visit',
 'diabetic_enrollees_age_65_to_75',  'average_diabetic_enrollees_hemoglobin_a1c_test',
 'average_diabetic_enrollees_eye_exam', 'average_diabetic_enrollees_blood_lipids_test',
 'average_female_enrollees_age_67_to_69', 'average_female_age_67_to_69_mammogram', 'beneficiaries_part_a_eligible',
 'leg_amputations_per_1000_enrollees', 'discharges_ambulatory_care_sensitive_conditions_per_1000_enrollees']]

In [25]:
# get X and y variables

y = data_all["HHI"]
X = features

### Train/Test/Split

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

X_train.head()

,Population,PovertyRate,beneficiaries_part_b,one_ambulatory_visit,diabetic_enrollees_age_65_to_75,average_diabetic_enrollees_hemoglobin_a1c_test,average_diabetic_enrollees_eye_exam,average_diabetic_enrollees_blood_lipids_test,average_female_enrollees_age_67_to_69,average_female_age_67_to_69_mammogram,beneficiaries_part_a_eligible,leg_amputations_per_1000_enrollees,discharges_ambulatory_care_sensitive_conditions_per_1000_enrollees
1190,6899123,13.282326,545956.0,78.3,58138.0,86.6,68.4,80.0,45223.0,62.8,598856.0,0.61,44.2
561,9886095,16.447161,940574.0,79.8,126208.0,85.5,65.6,79.9,76285.0,65.6,990373.0,0.66,64.5
631,5982413,14.526513,578710.0,79.0,72934.0,85.6,67.1,79.7,48778.0,62.2,620963.0,0.62,64.7
1125,624958,10.842489,80820.0,75.2,7810.0,88.9,71.3,78.9,7095.0,70.3,86153.0,0.42,51.1
1135,625904,11.323302,80820.0,75.2,7810.0,88.9,71.3,78.9,7095.0,70.3,86153.0,0.42,51.1


### MinMaxScaler

In [29]:
# Import dependencies


# scale the data
X_scaler =  StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)


print(encoded_y_test)

[ 44  26  21 154  66 165 145 199  21 209  52 225  39 142  64 152   2 156
  92 170 219  47 121 208 155  32 116  76 203  75   0  33 195  74 158 233
  19 188  49  15 225 149 202 137 175 204  63  81  79 189  48 101 147 246
  90 210 240  50  70 164 110 187  37  14 178 232 218 107 124 188 231 190
 153   4 186 201 189  71 227 221 118  59  83 130 111 166  54 241 212  11
  30 248  89  25 158 182  65  17 113  85 192  89 249 229  91 150  84  73
  17 132 208 228 216 205 106 161 200  13 234  13 102 102 180  51 198  63
  46 185 197  23 152 132 168 247  92  22  78 151 135 150 179 215  69 239
  48 141  36 144 175   6 160 226   3 103 172 163  10  57 196 131 217  58
 104 167  77 206  88 140  24 143 245 190 174  37  18 133 253 184 207  34
  54 169 239 203  27  52 214 234 148 242  62 109   8 129 130 252 236 139
 193 119 194 129 211 244 138  56 146 250 131 117 214 251 161 132  50  80
  14 181  41  55  29  72 235  40 167 121  58 134 100  97  98 173   8  12
   7  20 108 176 171 230 204 120  95 223  99  38 23

### Training the model

In [40]:
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train_scaled, encoded_y_train)
predictions = model.predict(X_test_scaled)
model

SVC(kernel='linear')

In [41]:
print(f"Training Data Score: {model.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, encoded_y_test)}")

Training Data Score: 0.4100418410041841
Testing Data Score: 0.04075235109717868


In [58]:
# HHI = data_all["HHI"].to_list()
# HHI = data_all["HHI"].array
# HHI

In [57]:
# Calculate classification report

from sklearn.metrics import classification_report
print(classification_report(encoded_y_test, predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         2
           9       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         1
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         1
          13       0.00      0.00      0.00         2
          14       0.00      0.00      0.00         2
          15       0.00      0.00      0.00         1
          16       0.00      0.00      0.00         1
          17       0.00    

C:\Users\me\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Feature Importance

In [11]:
HHI = data_all["HHI"]

In [12]:
target = data_all["HHI"]
target_names = [HHI]

In [13]:
data = data_all.drop("HHI", axis=1)
data = data.drop("state_name", axis=1)
feature_names = data.columns

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [15]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.9717868338557993

In [16]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9184952978056427

In [17]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.21425193866802192, 'PovertyRate'),
 (0.2013026594009392, 'Population'),
 (0.16272836167649388, 'year_x'),
 (0.043090848112651826, 'State'),
 (0.04075452092572989, 'state_id'),
 (0.040392320439636774, 'beneficiaries_part_b'),
 (0.03841466198531591, 'average_diabetic_enrollees_eye_exam'),
 (0.03691764223129977, 'diabetic_enrollees_age_65_to_75'),
 (0.03291177696101026, 'one_ambulatory_visit'),
 (0.0322203165766288, 'average_female_age_67_to_69_mammogram'),
 (0.030826215850074237, 'average_diabetic_enrollees_blood_lipids_test'),
 (0.030403117975833897, 'average_female_enrollees_age_67_to_69'),
 (0.027112576136348626, 'average_diabetic_enrollees_hemoglobin_a1c_test'),
 (0.026377064933637284, 'beneficiaries_part_a_eligible'),
 (0.018042308263381428,
  'discharges_ambulatory_care_sensitive_conditions_per_1000_enrollees'),
 (0.017105890089835922, 'leg_amputations_per_1000_enrollees'),
 (0.007147779773160423, 'year_y')]

### Tune Model Parameters using GridSearch (not working)

#### Grid selection test

In [ ]:
target = data["HHI"]
target_names = ["negative", "positive"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [ ]:
# Support vector machine linear classifier
from sklearn.svm import SVC 
model = SVC(kernel='linear')

In [ ]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10],
              'gamma': [0.0001, 0.001, 0.01]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
# List the best parameters for this dataset
print(grid.best_params_)

In [ ]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test)

In [ ]:
print('Test Acc: %.3f' % grid.score(X_test, y_test))

In [ ]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=["blue", "red"]))

#### Grid selection test2

In [ ]:
# Split data into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Create the SVC Model. Can modify parameters to get the best score
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters
grid.fit(X_train, y_train)

#### Grid selection test3

In [59]:
# Create the GridSearchCV model

from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [60]:
# Train the model with GridSearch
grid.fit(X_train_scaled, encoded_y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] C=1, gamma=0.0001 ...............................................


C:\Users\me\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................... C=1, gamma=0.0001, score=0.047, total=   0.4s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV] ................... C=1, gamma=0.0001, score=0.052, total=   0.5s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.8s remaining:    0.0s


[CV] ................... C=1, gamma=0.0001, score=0.058, total=   0.4s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.042, total=   0.4s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.084, total=   0.5s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ................... C=1, gamma=0.0005, score=0.047, total=   0.5s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ................... C=1, gamma=0.0005, score=0.052, total=   0.5s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ................... C=1, gamma=0.0005, score=0.058, total=   0.5s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ................... C=1, gamma=0.0005, score=0.042, total=   0.5s
[CV] C=1, gamma=0.0005 ...............................................
[CV] .

[CV] .................. C=50, gamma=0.0001, score=0.281, total=   0.4s
[CV] C=50, gamma=0.0001 ..............................................
[CV] .................. C=50, gamma=0.0001, score=0.372, total=   0.4s
[CV] C=50, gamma=0.0001 ..............................................
[CV] .................. C=50, gamma=0.0001, score=0.330, total=   0.5s
[CV] C=50, gamma=0.0001 ..............................................
[CV] .................. C=50, gamma=0.0001, score=0.319, total=   0.4s
[CV] C=50, gamma=0.0001 ..............................................
[CV] .................. C=50, gamma=0.0001, score=0.351, total=   0.6s
[CV] C=50, gamma=0.0005 ..............................................
[CV] .................. C=50, gamma=0.0005, score=0.281, total=   0.4s
[CV] C=50, gamma=0.0005 ..............................................
[CV] .................. C=50, gamma=0.0005, score=0.372, total=   0.6s
[CV] C=50, gamma=0.0005 ..............................................
[CV] .

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:   37.8s finished


GridSearchCV(estimator=SVC(kernel='linear'),
             param_grid={'C': [1, 5, 10, 50],
                         'gamma': [0.0001, 0.0005, 0.001, 0.005]},
             verbose=3)

In [61]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 50, 'gamma': 0.0001}
0.3305955497382199


In [62]:
# save the model

import joblib
filename = 'svm_model.sav'
joblib.dump(model, filename)

['svm_model.sav']